# 人工衛星LiDAR画像でピラミッドを見つけ出す('24.06.02)

## LiDAR撮影で見つけ出されたジャングルに隠れたピラミッド

「グアテマラ北部のジャングルの下に隠れていた古代マヤ文明の建造物が新しいレーザー技術によって発見された」という[記事](https://www.cnn.co.jp/style/architecture/35115470.html)を読んだ。
航空機からのLiDAR撮影で、高さ約２７メートルのピラミッドなどが発見されたという。
LiDARというのは、照射した光の反射を撮影し、たとえば形状などを推定するために使われる。

ピラミッドの形状は、光学的には例えると、車のヘッドライト光を反射するキャッツアイ的な構造だ。
だから、ピラミッド構造は、LiDAR撮影画像には独特なパターンとして現れる。
そのため、LiDAR撮影画像に生じた模様を分析すれば、ピラミッド的な構造を検出することができる、というわけだ。


## Google Earth EngineとfoliumでLiDAR撮影画像/可視光地図を眺める

そこで、地球上のさまざまな場所を、地図・人工衛星可視光画像・LiDAR撮影で眺めることができるようにして、未発見のピラミッドを探すことができるようにしてみようと思う。
そのためのPythonコード、Google Earth EngineのAPIとfoliumを使って、地図・人工衛星可視光画像・LiDAR撮影で眺めるためのコードを書いて、実行してみる。

In [1]:
center=[29.975811646205145, 31.130714568679988]

In [2]:
import folium
import ee

# GEEの認証
ee.Authenticate()
ee.Initialize()

basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [3]:
# foliumマップにGEEを表示させる関数
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='<a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# 地図を作成
fmap = folium.Map(location=center, zoom_start=10)

basemaps['Google Maps'].add_to(fmap)
basemaps['Google Satellite Hybrid'].add_to(fmap)

# Sentinel-1
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate('2020-01-01', '2023-12-31') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
# ImageCollectionからモザイクを作成
mosaic = sentinel1.mosaic()

# 地図にレイヤーを追加
folium.Map.add_ee_layer = add_ee_layer
fmap.add_ee_layer(mosaic.select(['VH']), {'min': -25,'max': 5}, 'Sentinel-1 VH')
fmap.add_ee_layer(mosaic.select(['VV']), {'min': -25,'max': 5}, 'Sentinel-1 VV')
# コントロール部品を追加
fmap.add_child(folium.LayerControl(collapsed = False).add_to(map))

AttributeError: type object 'map' has no attribute 'add_child'

まずは、人工衛星から撮影した、エジプトのカイロ近くのピラミッド可視光画像を眺めてみる。
ピラミッドは、カイロの町近くにあって、こんな景色を眺めて暮らしてみたかった気もする。

%<img style="float:center;transform: rotate(0deg); height:10cm" src="./images/day_240601_pyramid_rgb.png" />

```{figure} ./images/day_240601_pyramid_rgb.png
---
height: 10cm
---
エジプトのカイロ近くのピラミッド（可視光画像）
```

次に、人工衛星から撮影した、LiDAR画像を眺めてみる。
ピラミッド的な構造、キャッツアイ的な構造に光が斜めから当たると、明暗模様が生まれる。
そういうことを知った上で下の画像を眺めると、ピラミッドが存在していることがわかる。

%<img style="float:center;transform: rotate(0deg); height:10cm" src="./images/day_240601_pyramid_lidar.png" />

```{figure} ./images/day_240601_pyramid_lidar.png
---
height: 10cm
---
エジプトのカイロ近くのピラミッド（LiDAR画像）
```


## エジプトから南米まで、世界中にあるピラミッド

今日は、未だ発見されていないピラミッドを見つけるための、スタートラインのコードを書いてた。
南米のグアテマラにあるピラミッド発見のニュースをきっかけに、アフリカとヨーロッパとアジアの中間に位置するエジプトのピラミッドを眺めると、世界中にピラミッドが点在していることを意識させられる。
そして、世界中のピラミッドを眺めに行ってみたいと思う。
